**Decsion Tree and Logistic Regression for Multi classification**

In [1]:
# every package has to install each time googlecolab runs jyputernote book.

!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=c676b6c8d3b360403b5f97ba1fba232e136b081c42f5ba05c9316b8e17dbfb3f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [6]:
# input the data csv file with the pathpath
dataset = spark.read.csv('/content/bezdekIris.data',inferSchema=True, header =True)\
.toDF("sep_len", "sep_wid", "pet_len", "pet_wid", "label")
dataset.show(5)


+-------+-------+-------+-------+-----------+
|sep_len|sep_wid|pet_len|pet_wid|      label|
+-------+-------+-------+-------+-----------+
|    4.9|    3.0|    1.4|    0.2|Iris-setosa|
|    4.7|    3.2|    1.3|    0.2|Iris-setosa|
|    4.6|    3.1|    1.5|    0.2|Iris-setosa|
|    5.0|    3.6|    1.4|    0.2|Iris-setosa|
|    5.4|    3.9|    1.7|    0.4|Iris-setosa|
+-------+-------+-------+-------+-----------+
only showing top 5 rows



In [10]:
dataset.select('label').distinct().show(10)
dataset.count()

+---------------+
|          label|
+---------------+
| Iris-virginica|
|    Iris-setosa|
|Iris-versicolor|
+---------------+



149

In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
vector_assembler = VectorAssembler(\
inputCols = ["sep_len","sep_wid","pet_len","pet_wid"],\
outputCol = "features")
df_temp = vector_assembler.transform(dataset)
df_temp.show()

+-------+-------+-------+-------+-----------+-----------------+
|sep_len|sep_wid|pet_len|pet_wid|      label|         features|
+-------+-------+-------+-------+-----------+-----------------+
|    4.9|    3.0|    1.4|    0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|    4.7|    3.2|    1.3|    0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|    4.6|    3.1|    1.5|    0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|    5.0|    3.6|    1.4|    0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
|    5.4|    3.9|    1.7|    0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|
|    4.6|    3.4|    1.4|    0.3|Iris-setosa|[4.6,3.4,1.4,0.3]|
|    5.0|    3.4|    1.5|    0.2|Iris-setosa|[5.0,3.4,1.5,0.2]|
|    4.4|    2.9|    1.4|    0.2|Iris-setosa|[4.4,2.9,1.4,0.2]|
|    4.9|    3.1|    1.5|    0.1|Iris-setosa|[4.9,3.1,1.5,0.1]|
|    5.4|    3.7|    1.5|    0.2|Iris-setosa|[5.4,3.7,1.5,0.2]|
|    4.8|    3.4|    1.6|    0.2|Iris-setosa|[4.8,3.4,1.6,0.2]|
|    4.8|    3.0|    1.4|    0.1|Iris-setosa|[4.8,3.0,1.4,0.1]|
|    4.3|    3.0|    1.1|    0.1|Iris-se

In [12]:
#remove unnecessary columns:
df = df_temp.drop('sep_len','sep_wid','pet_len','pet_wid')
df.show()

+-----------+-----------------+
|      label|         features|
+-----------+-----------------+
|Iris-setosa|[4.9,3.0,1.4,0.2]|
|Iris-setosa|[4.7,3.2,1.3,0.2]|
|Iris-setosa|[4.6,3.1,1.5,0.2]|
|Iris-setosa|[5.0,3.6,1.4,0.2]|
|Iris-setosa|[5.4,3.9,1.7,0.4]|
|Iris-setosa|[4.6,3.4,1.4,0.3]|
|Iris-setosa|[5.0,3.4,1.5,0.2]|
|Iris-setosa|[4.4,2.9,1.4,0.2]|
|Iris-setosa|[4.9,3.1,1.5,0.1]|
|Iris-setosa|[5.4,3.7,1.5,0.2]|
|Iris-setosa|[4.8,3.4,1.6,0.2]|
|Iris-setosa|[4.8,3.0,1.4,0.1]|
|Iris-setosa|[4.3,3.0,1.1,0.1]|
|Iris-setosa|[5.8,4.0,1.2,0.2]|
|Iris-setosa|[5.7,4.4,1.5,0.4]|
|Iris-setosa|[5.4,3.9,1.3,0.4]|
|Iris-setosa|[5.1,3.5,1.4,0.3]|
|Iris-setosa|[5.7,3.8,1.7,0.3]|
|Iris-setosa|[5.1,3.8,1.5,0.3]|
|Iris-setosa|[5.4,3.4,1.7,0.2]|
+-----------+-----------------+
only showing top 20 rows



Now we should index labels, i.e., convert textual representation to a numeric
List item
List item
one with the help of StringIndexer. To do this, execute the following commands:

In [13]:
from pyspark.ml.feature import StringIndexer
l_indexer = StringIndexer(inputCol="label", outputCol="labelIndex")
df = l_indexer.fit(df).transform(df)

In [14]:
df.select('label','labelIndex').distinct().show(3)

+---------------+----------+
|          label|labelIndex|
+---------------+----------+
|Iris-versicolor|       0.0|
| Iris-virginica|       1.0|
|    Iris-setosa|       2.0|
+---------------+----------+



Divide the data into train and test set(30% for test)

In [15]:
(trainingData, testData) = df.randomSplit([0.7, 0.3])

Multi Classification using Decsion Tree

In [22]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
dt = DecisionTreeClassifier(labelCol="labelIndex", featuresCol="features",impurity='entropy', maxDepth=4,seed=1234)
model = dt.fit(trainingData)
predictions = model.transform(testData)

To estimate the accuracy of the prediction, the test error should be computed :

In [17]:
evaluator = MulticlassClassificationEvaluator(\
labelCol="labelIndex", predictionCol="prediction",\
metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test accuracy =  " , accuracy)
print(model.toDebugString)

Test accuracy =   0.9347826086956522
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_ec566bc58ec8, depth=4, numNodes=11, numClasses=3, numFeatures=4
  If (feature 2 <= 2.45)
   Predict: 2.0
  Else (feature 2 > 2.45)
   If (feature 3 <= 1.75)
    If (feature 2 <= 4.95)
     If (feature 0 <= 4.95)
      Predict: 1.0
     Else (feature 0 > 4.95)
      Predict: 0.0
    Else (feature 2 > 4.95)
     If (feature 3 <= 1.65)
      Predict: 1.0
     Else (feature 3 > 1.65)
      Predict: 0.0
   Else (feature 3 > 1.75)
    Predict: 1.0



Multiclassification using Logisitc Regression

In [19]:
# this is code for multiple classification using logistic Regression
from pyspark.ml.classification import OneVsRest
from pyspark.ml.classification import LogisticRegression
train, test = df.randomSplit([0.7, 0.3], seed = 2018)
lr = LogisticRegression(maxIter=100, \

                        featuresCol="features", \

                        labelCol='labelIndex')
ovr = OneVsRest(classifier=lr, \
                labelCol='labelIndex', \
                featuresCol='features')
#from pyspark.ml import Pipeline
#pipeline_ovr = Pipeline(stages=[vecAssembler, stdScaler, ovr])
#pipelineModel_ovr = pipeline_ovr.fit(trainDF)

ovrModel = ovr.fit(train)
predictionsovr = ovrModel.transform(test)

In [20]:
evaluator = MulticlassClassificationEvaluator(\
labelCol="labelIndex", predictionCol="prediction",\
metricName="accuracy")
accuracy = evaluator.evaluate(predictionsovr)
print("Test accuracy =  " , accuracy)

Test accuracy =   0.9361702127659575
